# Roboflowからのデータのダウンロード

In [ ]:
!pip install roboflow | tail -n 1
ROBOFLOW_API_KEY = ""
from roboflow import Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("bismillah-eoen6").project("train2_tugas-akhir")
version = project.version(9)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Train2_Tugas-Akhir-9 in yolov11:: 100%|██████████| 30714/30714 [00:08<00:00, 3608.68it/s]


# directory構成の整備

In [2]:
!mv Train2_Tugas-Akhir-9 dataset

In [3]:
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [4]:
!mkdir ./dataset/images
!mkdir ./dataset/labels
!mv ./dataset/train/images/* ./dataset/images
!mv ./dataset/valid/images/* ./dataset/images
!mv ./dataset/test/images/* ./dataset/images
!mv ./dataset/train/labels/* ./dataset/labels
!mv ./dataset/valid/labels/* ./dataset/labels
!mv ./dataset/test/labels/* ./dataset/labels
!rm -r ./dataset/train
!rm -r ./dataset/valid
!rm -r ./dataset/test

# データローダーの定義

In [5]:
from PIL import Image
import pandas as pd
import matplotlib.patches as patches

In [6]:
class LabeledImage:
    def __init__(self, name):
        self.name = name
        self.image_path = f'./dataset/images/{name}.jpg'
        self.label_path = f'./dataset/labels/{name}.txt'

        if not os.path.exists(self.image_path):
            raise FileNotFoundError(f"Image not found: {self.image_path}")
        if not os.path.exists(self.label_path):
            raise FileNotFoundError(f"Label file not found: {self.label_path}")

    def show_image(self):
        img = Image.open(self.image_path)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f'Image: {self.name}.jpg')
        plt.show()

    def show_labels(self):
        df = self._load_labels_dataframe()
        print(f'Labels for {self.name}.txt (YOLO format):')
        display(df)

    def show_all(self, draw_boxes=False):
        """Display image and label table side by side, with optional bounding boxes."""
        df = self._load_labels_dataframe()
        df_T = df.T

        fig, axs = plt.subplots(1, 2, figsize=(13, 5), gridspec_kw={'width_ratios': [1, 1.1]})

        # Load image
        img = Image.open(self.image_path)
        axs[0].imshow(img)
        axs[0].axis('off')
        axs[0].set_title(f'Image: {self.name}.jpg', fontsize=12)

        # Optionally draw bounding boxes
        if draw_boxes and not df.empty and 'x_center' in df.columns:
            width, height = img.size
            for _, row in df.iterrows():
                try:
                    xc = float(row['x_center']) * width
                    yc = float(row['y_center']) * height
                    w = float(row['width']) * width
                    h = float(row['height']) * height
                    xmin = xc - w / 2
                    ymin = yc - h / 2
                    rect = patches.Rectangle((xmin, ymin), w, h,
                                             linewidth=2, edgecolor='red', facecolor='none')
                    axs[0].add_patch(rect)
                    axs[0].text(xmin, ymin - 5, str(row['Label']),
                                color='red', fontsize=10, backgroundcolor='white')
                except Exception as e:
                    continue  # skip invalid rows

        # Right: label table
        axs[1].axis('off')
        axs[1].set_title(f'Labels: {self.name}.txt', fontsize=12)

        table = axs[1].table(
            cellText=df_T.values,
            rowLabels=df_T.index,
            colLabels=df_T.columns,
            loc='center',
            cellLoc='center',
            colLoc='center',
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.1, 1.3)

        plt.tight_layout()
        plt.show()

    def _load_labels_dataframe(self):
        """Load labels from YOLO-format text file into a DataFrame."""
        with open(self.label_path, 'r') as f:
            lines = f.readlines()

        data = []
        if lines:
            for i, line in enumerate(lines, start=1):
                parts = line.strip().split()
                if len(parts) == 5:
                    label, x, y, w, h = parts
                    data.append({
                        'Line': i,
                        'Label': label,
                        'x_center': x,
                        'y_center': y,
                        'width': w,
                        'height': h
                    })
                else:
                    data.append({
                        'Line': i,
                        'Label': '[invalid]',
                        'x_center': '',
                        'y_center': '',
                        'width': '',
                        'height': line.strip() or '[empty]'
                    })
        else:
            data.append({
                'Line': '-',
                'Label': '[no label]',
                'x_center': '', 'y_center': '', 'width': '', 'height': ''
            })

        return pd.DataFrame(data)

# 画像の切り出し

正解画像のファイル名は以下のテキスト`.jpg`

In [8]:
clean_files = [
    '1-150-_jpg.rf.5f11b2a20acd4ebe0e7badd6d565bda3',
 '1-150-_jpg.rf.c8ece3f6d7c0960e9308edd6a891750a',
 '1-150-_jpg.rf.de3b439e034b9be3bb00b98b9fdaf60c',
 '10-155-_jpg.rf.7d5e9f2a92f7f91e8688c27fd0e193a0',
 '11-34-_jpg.rf.cfbd37c000ab152b709e0fdd976e3964',
 '11-35-_jpg.rf.5e91dab7b4a28387054bebdbd9f7c8fd',
 '11-35-_jpg.rf.94b4877e7e9bb063133edf193ba5eac5',
 '11-35-_jpg.rf.b726d53961952b67c10f35a8723a2ca9',
 '12-129-_jpg.rf.0e554f015321aad918e1d6d0d6a051d0',
 '12-129-_jpg.rf.8187998bd09edc5f84d9b4057a50deaa',
 '12-129-_jpg.rf.ebc0d4f4affdef90012587ae9573b74e',
 '12-130-_jpg.rf.3d946ab5db54a68a8f4013c47a4f3e5e',
 '12-130-_jpg.rf.c9934fa36ef13f2c10405c2f776c9546',
 '12-130-_jpg.rf.d1a6bc07258ba393c8c9168dfc77f34f',
 '13-97-_jpg.rf.0d6f52913ec0ab12da95b7ea979ddd6b',
 '13-97-_jpg.rf.645207a678f28e28edda314f8ead0438',
 '13-97-_jpg.rf.c748809c212aabd5a2c48a59c665bbd6',
 '13-98-_jpg.rf.85a2ae11f1662b327df1c9533bc6654f',
 '13-98-_jpg.rf.dd533b457d8a49ab7b281c18e3e6e22c',
 '13-98-_jpg.rf.e560786131cb4fc744c75c0e6cb4af25',
 '13-99-_jpg.rf.2ad7fcb19f68a381f0ff50e8e1f09ff2',
 '13-99-_jpg.rf.44c0fea161c5d21c54e3e7fb08371e0c',
 '13-99-_jpg.rf.d9f048e793727557fad8b0a19534deb4',
 '14-147-_jpg.rf.77318f71cf80b6330c03579da9d84c38',
 '14-147-_jpg.rf.7749269ee62ae45ce9c7658fb8e9efa1',
 '14-147-_jpg.rf.bd2ca290e2e08bffba20f4855a1214bd',
 '15-70-_jpg.rf.c88b74eb4d450b5cb4edfd9f7a443b86',
 '15-71-_jpg.rf.2301d060027900713e84e3b203699b48',
 '15-71-_jpg.rf.99005aa8275854c22f7dc65b26912348',
 '15-71-_jpg.rf.d70aff7202586176df4ec9666876488e',
 '16-94-_jpg.rf.56357dce4ee5f063a3c683b126247aa0',
 '16-94-_jpg.rf.88df752b59acdea3a95efbd10e03bc66',
 '16-94-_jpg.rf.bc70873ed351992c70310c70a16a4a1c',
 '16-95-_jpg.rf.68c32b07994e1a4bf50b970e431f0fec',
 '16-95-_jpg.rf.e102d7989127afcad8bf73f486ce0ff6',
 '16-95-_jpg.rf.e6e217456a13c25f4f7a32c27e46e2de',
 '17-155-_jpg.rf.75ebf615e81e6f07eee1431faf2167f8',
 '18-1-_jpg.rf.c0f43eb921fad2ef5c69b8b77c6263e7',
 '18-1-_jpg.rf.d5230b5e93c7282d9724d9a38c95b4de',
 '18-1-_jpg.rf.f53ca28e6c0b8214e283e2da20e09135',
 '18-117-_jpg.rf.14a3358f013ba1ed908d27288914bcf2',
 '18-117-_jpg.rf.14b35f51e04482ccae7f175420c04a1c',
 '18-117-_jpg.rf.4d904ba515c871701e6f91c3f31621e8',
 '19-155-_jpg.rf.6316e46ddaf7e3abcf4b6afc6a779b8f',
 '20-153-_jpg.rf.1dfec0530b947373b30f923944570be6',
 '20-153-_jpg.rf.eca5c84bd9ed6efe98c84a3da6c4fc86',
 '20-153-_jpg.rf.fa39d487f27fb52054215ee55b132642',
 '20-154-_jpg.rf.089e280d9fde8ddec599e7ff374b6d20',
 '20-154-_jpg.rf.4dd3570e532cda28f65bb3dabf2c87e7',
 '20-154-_jpg.rf.e0af6a9c176f2c41bab00e40e15752e0',
 '20-155-_jpg.rf.3f360fde47270ddf034fcc355c4ac77b',
 '22-1-_jpg.rf.d8a621d5af8686757a4c1c7ca62bae51',
 '22-1-_jpg.rf.d994d67b11f1d06a0f9d5042364ecd61',
 '22-1-_jpg.rf.fed74b8bc8b0e6e9cf78efc636d85d24',
 '22-153-_jpg.rf.16394d2249651a52d5bf843bd4d98c95',
 '22-153-_jpg.rf.61d26ae26fea6d89d03724597b670324',
 '22-153-_jpg.rf.ac58017ba881b6d1e15b8dd0645bf343',
 '22-154-_jpg.rf.92b1d1bc526d561a1c8f37a633ac79e4',
 '22-154-_jpg.rf.bd0fecb011f93a3649b928998f6b1fc3',
 '22-154-_jpg.rf.c72274b2e788771bb2e694e09d918e9e',
 '22-155-_jpg.rf.057415441b9b324824995abd87ec7f97',
 '23-3-_jpg.rf.2a8bd16b25d9c7a198c5413d2ff82c14',
 '23-3-_jpg.rf.9377c0246453339bad2e48d0d5d39235',
 '23-3-_jpg.rf.cf3d2be63975a2a9ecd62fe8cb7aeacb',
 '23-31-_jpg.rf.dda7c04b0cace3de3f8a71b8be8c0349',
 '23-4-_jpg.rf.5fbad10fc3187445a711fe3bc34dab86',
 '23-4-_jpg.rf.f195ec628663a042e0731bff4ef348cd',
 '23-4-_jpg.rf.fd63a301826caaf10d4d227019a21374',
 '23-50-_jpg.rf.7a31eef97a039c9923460b2daa56fce4',
 '24-152-_jpg.rf.c4ad056b105f3af0d7ff97b34521e016',
 '24-153-_jpg.rf.6059d2a1a5cadb26febd809685910230',
 '24-153-_jpg.rf.ebb71f73f88ba67949aa768de029161e',
 '24-153-_jpg.rf.fa9c6e1fd2f23b29c8091c1523264d75',
 '24-155-_jpg.rf.46d0dddc5c6d21d2843a333a975fcd27',
 '24-155-_jpg.rf.49b703f1c02c22441e480ebac8055ba5',
 '24-155-_jpg.rf.63bb044f3e5a6e28e84db4f8347be188',
 '25-155-_jpg.rf.2e78dca252d4aeb12333245576977051',
 '25-155-_jpg.rf.2eb98b31995e905c9b441ebd7b6556ba',
 '25-155-_jpg.rf.b147b9b11cb210714d77d22a4d97e309',
 '26-33-_jpg.rf.09c07fb84cf5eaa5339d547ecb194142',
 '26-33-_jpg.rf.b3657640559acc65cee3cd42f0e8c590',
 '26-33-_jpg.rf.b5c8907f3b7b57e96be09ac49c479e44',
 '27-78-_jpg.rf.40768483b04121a9aad6a165b94be2d2',
 '27-78-_jpg.rf.6f62807bc3f411cdfb6ab1eb158b6ad5',
 '27-78-_jpg.rf.ddc36e0cd06160b1db43a5edd3e034aa',
 '27-79-_jpg.rf.1e46af57e6c0b19407297c3b4e920efd',
 '27-79-_jpg.rf.c14d8061be6bb183d7c95ab2d3b5c84b',
 '27-79-_jpg.rf.e3bd59a4520e5ea203d1a9db16cff598',
 '28-154-_jpg.rf.61635911cafd5d3442afd7d76f03a860',
 '28-154-_jpg.rf.9d76f8dc7db754e0f2ee7b4bb73d3dd9',
 '28-154-_jpg.rf.caca6954ccb3cc27144e893a977d5b85',
 '28-155-_jpg.rf.43f8545917fdbf71bae04ada5cb36c8c',
 '28-155-_jpg.rf.77b0798e35f111c6d9b5c6b6f93b8069',
 '28-155-_jpg.rf.e70948e24005e330f2434351988e3680',
 '29-151-_jpg.rf.4bef55881d5e2da8b9d3fa6810f399ef',
 '29-151-_jpg.rf.a160839da23b9ea5621fa27298d552e8',
 '29-151-_jpg.rf.b234b542d1be809821d32be6037736c6',
 '29-152-_jpg.rf.5c2026bcb44f2b7d634281776451a5a5',
 '29-153-_jpg.rf.299fc4e17eaeb4ca82bdbd7513f6493a',
 '29-153-_jpg.rf.cd4ce3e7b284235b28128495a022caea',
 '29-153-_jpg.rf.d78d22aafe94260b7d0d0ab7a28241da',
 '29-154-_jpg.rf.74032768d2992d9d356a81385a06348c',
 '29-154-_jpg.rf.917a0f003c2b82278be3836763dfa1b7',
 '29-154-_jpg.rf.9b6f84f8f3d7b6c707221f5e31ea9004',
 '29-155-_jpg.rf.45fd06a3cc090c55e7d90f1b4ab40ab1',
 '29-155-_jpg.rf.8fcb6e8ee7fb2f7ddfc5899bee0d6427',
 '29-155-_jpg.rf.b0628c9b1aa5dfd73d2512c7a9cd7037',
 '30-129-_jpg.rf.dfe6e6ebbed12ebdbd02d924c1d0691c',
 '30-42-_jpg.rf.152cf7b51066006f37f268de21d7aa82',
 '30-42-_jpg.rf.17f5379b2d1757f0572ddbc9e898157b',
 '30-42-_jpg.rf.3dab1d2127534788afe846a36d54f831',
 '4-155-_jpg.rf.38924cd443b8e9487af363b787bd5f65',
 '4-71-_jpg.rf.d2a79d20d5794bf68182829592a5956d',
 '4-72-_jpg.rf.46d134b430511a8f459fc82e1d3ed3de',
 '4-72-_jpg.rf.52ddcdc4669d10a02bde8dcde930ed4b',
 '4-72-_jpg.rf.7670d5b9dc13e33fba8f2747c9e5fbee',
 '4-74-_jpg.rf.15305a2c07b08267c2fb8ad3cb5b38a1',
 '4-74-_jpg.rf.35b7a11f358e730eb87e383753ed7e10',
 '4-74-_jpg.rf.f7d80d659a698e70fd02c2e64a780d67',
 '5-155-_jpg.rf.5824b76914dbd4441382c9c7c2212b1a',
 '5-155-_jpg.rf.58790b69a43328a9dc9bb14694fac127',
 '5-155-_jpg.rf.f1115c210a1f19bf554383b8e5269a6a',
 '6-143-_jpg.rf.751a998d3a3b729795d4ce7e14b479a5',
 '6-144-_jpg.rf.04930606a9baa75377535a5f501532b4',
 '6-144-_jpg.rf.8f43fbf87cabfc1b2ae6b84dbb2f8f7b',
 '6-144-_jpg.rf.a2cbe9c4f405f2f4dbf366bf8c1d2381',
 '7-155-_jpg.rf.1d3ca7969bcca01b6dc4b96fc96e5d4b',
 '7-155-_jpg.rf.91dd3eb1ae8765da1eaa7312fd09a71d',
 '7-155-_jpg.rf.b2a89095493815b0c60f5e4594484ea1',
 '8-143-_jpg.rf.77c2040c4e531ef3fd9e19e28de7afd5',
 '8-143-_jpg.rf.afdafa39359261c703e1f799faeaf22e',
 '8-143-_jpg.rf.b1cc4d46409dff8378758e53dd13950d',
 '8-155-_jpg.rf.d925dee709ab68e92920703b220d3735',
 '8-155-_jpg.rf.e2b34c5d063aab4a21ce4721f6a0429b',
 '8-155-_jpg.rf.e84568ea7207254818cb94d5fabdf799',
 '9-87-_jpg.rf.0c729764a312b809e696680268980eb5',
 '9-88-_jpg.rf.185868435de8aabeeb5dd492ba2487d6',
 '9-88-_jpg.rf.1d8b7d8cd85438de35fd8ac4b7858cbd',
 '9-88-_jpg.rf.ef9d56588a0bbda7bb7d13d8503deb15',
 'c-307-_jpg.rf.6619d127a69515f2e6c37f74af5313e5',
 'c-307-_jpg.rf.8ef9b371d7ef586dd213758e2bd80031',
 'c-307-_jpg.rf.a4f3cb66cf4e95aad4d73181b6e028dc',
 'c-308-_jpg.rf.30adb738e31bebede68f2e157716b890',
 'c-308-_jpg.rf.3c8727066b477f1013e110bc78b12892',
 'c-308-_jpg.rf.e2b3091648f07423d19de3ca6899298a',
 'download-5-Copy_png.rf.fcfd52afe6b508265bba2bfe69e0afc4',
 'W1-2-L-1-_png_jpg.rf.51ee32eb5ec407f05dcc0167f30f9d6e',
 'W1-2-L-2-_png_jpg.rf.fcb670d02cffd184da2ee976080e4be7',
 'W1-2-L-3-_png_jpg.rf.4efcc824837fec678e9e94cb9472a546',
 'W1-2-L-4-_png_jpg.rf.d0d036ef5a5f0b96fda7fe4b6a498bde',
 'W1-2-L-5-_png_jpg.rf.26287c88f4fa25a197fca07ace161ea9',
 'W1-2-L-6-_png_jpg.rf.0acfa416def8a30d34864a3df67b2b6d',
 'W1-2-R-1-_png_jpg.rf.3ebba31fd036ff48ef9042711b93dd07',
 'W1-2-R-2-_png_jpg.rf.aed3db52fc5835b0af153da637c3c5b0',
 'W1-2-R-3-_png_jpg.rf.486e172dbdd36d1b7172ba71e545bb9e',
 'W1-2-R-4-_png_jpg.rf.97bc827c51915005d45e472402b3b763',
 'W1-2-R-5-_png_jpg.rf.1e4bd84f6eddd3b3c12df79defa4eace',
 'W1-2-R-6-_png_jpg.rf.c5fbedb703261b849025b543863d7e0a',
 'W1-3-R-1-_png_jpg.rf.0542686ce10b88cdfa44555b75b47cb6',
 'W1-3-R-2-_png_jpg.rf.72898e5662ccde3f98966c6d923ef208',
 'W1-3-R-3-_png_jpg.rf.49d1e4b9821af04e8e09e02d82969c38',
 'W1-3-R-4-_png_jpg.rf.5464a42710d3ae8e56f13ed63a3974cb',
 'W1-3-R-5-_png_jpg.rf.24c4b70fb7925f4be88c19948e080cbe',
 'W1-3-R-6-_png_jpg.rf.265b6dec361f96c5a691c364fceb1eb6',
 'W1-5-L-1-_png_jpg.rf.c3592ae9a2bb46f76808ecb58bb41bee',
 'W1-5-L-2-_png_jpg.rf.b76bfc22e57ff57c3303ab25610c3c03',
 'W1-5-L-3-_png_jpg.rf.248e26dabde1c784352e1887f314bc5d',
 'W1-5-L-4-_png_jpg.rf.5cdd0c5e7ffe31075a8e4d6514b705f7',
 'W1-5-L-6-_png_jpg.rf.ef7a46c62c5f35e2e5e28a7708d77a70',
 'W1-5-R-1-_png_jpg.rf.d466c4450e6e3ca1b47f05d94dad62b1',
 'W1-5-R-2-_png_jpg.rf.01f2c329ef78acaa8e30f2b62d4101a2',
 'W1-5-R-3-_png_jpg.rf.1c39544bb1cbcddf9eb38ba6ada1a23a',
 'W1-5-R-4-_png_jpg.rf.f39ebf643225334038f06e41e1e8f859',
 'W1-5-R-5-_png_jpg.rf.2d214a8a337a0983538727e8ab8e247c',
 'W1-5-R-6-_png_jpg.rf.5a29930097e9cc0db140002376f75792']

In [9]:
clean_files = set(clean_files)

In [10]:
from tqdm.notebook import tqdm
import os
import cv2
import glob
import shutil
import random
from collections import defaultdict

きれいな標識画像は，`cleanimage` ディレクトリ下のラベル名のディレクトリ内に格納．

それ以外は `train/valid/test` に $7:2:1$ の比に分割して対応するディレクトリの，ラベル名のディレクトリ下に配置．

In [ ]:
# 設定
image_dir = "./dataset/images"
label_dir = "./dataset/labels"
output_base = "./dataset"
splits = ['train', 'val', 'test']
split_ratio = [0.7, 0.2, 0.1]
output_size = (100, 100)
class_digits = 2
id_digits = 3

# ディレクトリ作成
for split in splits:
    os.makedirs(os.path.join(output_base, split), exist_ok=True)

# データ収集
image_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")))
label_paths = {os.path.splitext(os.path.basename(p))[0]: p for p in glob.glob(os.path.join(label_dir, "*.txt"))}

# 分割
random.seed(42)
random.shuffle(image_paths)
split_points = [int(len(image_paths) * sum(split_ratio[:i+1])) for i in range(len(split_ratio))]
split_image_lists = [
    image_paths[:split_points[0]],
    image_paths[split_points[0]:split_points[1]],
    image_paths[split_points[1]:]
]

# クラスごとのカウンター
class_id_counter = defaultdict(int)
clean_id_counter = defaultdict(int)

# データ処理
for split, images in zip(splits, split_image_lists):
    for image_path in tqdm(images):
        base = os.path.splitext(os.path.basename(image_path))[0]
        label_path = label_paths.get(base)
        if not label_path:
            continue

        img = cv2.imread(image_path)
        height, width = img.shape[:2]

        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, x_center, y_center, w, h = map(float, parts)
            cls = int(cls)
            cls_str = str(cls + 1).zfill(class_digits)

            x1 = int((x_center - w / 2) * width)
            y1 = int((y_center - h / 2) * height)
            x2 = int((x_center + w / 2) * width)
            y2 = int((y_center + h / 2) * height)

            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(width, x2)
            y2 = min(height, y2)

            cropped = img[y1:y2, x1:x2]
            if cropped.size == 0:
                continue

            resized = cv2.resize(cropped, output_size)

            class_id_counter[cls_str] += 1
            file_id = str(class_id_counter[cls_str]).zfill(id_digits)

            if base in clean_files:
                clean_id_counter[cls_str] += 1
                file_id = str(clean_id_counter[cls_str]).zfill(id_digits)
                save_dir = os.path.join(output_base, "cleanimage", cls_str)
                filename = f"clean{cls_str}{file_id}.jpg"
            else:
                class_id_counter[cls_str] += 1
                file_id = str(class_id_counter[cls_str]).zfill(id_digits)
                save_dir = os.path.join(output_base, split, cls_str)
                filename = f"img{cls_str}{file_id}.jpg"

            os.makedirs(save_dir, exist_ok=True)
            cv2.imwrite(os.path.join(save_dir, filename), resized)

In [16]:
!ls -l ./dataset/cleanimage/ | grep ^d | wc -l

34


# ラベル名の翻訳

In [17]:
labelname_en = [
    "First Aid Station",
    "Children Area",
    "Winding Road Right",
    "Winding Road Left",
    "Stop",
    "No Right Turn",
    "No Left Turn",
    "No Stopping",
    "No Entry",
    "No Overtaking",
    "No Parking",
    "No U-Turn",
    "Church",
    "Caution",
    "Roundabout",
    "Bicycle Lane",
    "Max Speed 30",
    "Max Speed 40",
    "Left Lane",
    "Traffic Light",
    "Weight Limit 10t",
    "Mosque",
    "Bus Stop",
    "Pedestrian Crossing",
    "Railroad Crossing",
    "Crosswalk Required",
    "T-Junction Priority",
    "T-Junction Right Priority",
    "T-Junction Left Priority",
    "T-Junction Left",
    "Crossroad",
    "Choose One Lane",
    "Speed Bump",
    "Gas Station",
    "U-Turn",
    "Hospital",
    "Parking Area",
    "Double Curve Right",
    "Double Curve Left",
    "Right Turn"
]

In [18]:
labelname_ja = [
    "応急処置所",
    "子ども注意",
    "右カーブあり",
    "左カーブあり",
    "一時停止",
    "右折禁止",
    "左折禁止",
    "停止禁止",
    "進入禁止",
    "追い越し禁止",
    "駐車禁止",
    "転回禁止",
    "教会",
    "注意",
    "ラウンドアバウト",
    "自転車専用通行帯",
    "最高速度30km/h",
    "最高速度40km/h",
    "左車線",
    "信号機あり",
    "重量制限10t",
    "モスク",
    "バス停",
    "横断歩道",
    "踏切注意",
    "横断指示",
    "優先T字路",
    "右優先T字路",
    "左優先T字路",
    "左方向T字路",
    "十字路",
    "いずれかの車線選択",
    "減速帯",
    "ガソリンスタンド",
    "転回可",
    "病院",
    "駐車場",
    "二重カーブ右",
    "二重カーブ左",
    "右カーブ"
]

In [19]:
num_to_lab_en = {i: labelname_en[i] for i in range(len(labelname_en))}
num_to_lab_ja = {i: labelname_ja[i] for i in range(len(labelname_ja))}